<center>
    <h1>Nader Mohamed Elhadedy</h1>
</center>

In [1]:
from pymongo import MongoClient

# Connecting to MongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017")

# Helper Functions

In [3]:
def print_cursor(cursor):
    c = 0
    for i in cursor:
        print()
        print(i)
        c += 1
    print()
    print("#Results:", c)

# Dataset 1

In [19]:
db = client.universities

In [20]:
client.drop_database("universities")

In [21]:
db.universities.insert_many([
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "USAL",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6722512,17, 40.9607792 ]
  },
  "students" : [
    { "year" : 2014, "number" : 24774 },
    { "year" : 2015, "number" : 23166 },
    { "year" : 2016, "number" : 21913 },
    { "year" : 2017, "number" : 21715 }
  ]
},
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "UPSA",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6691191,17, 40.9631732 ]
  },
  "students" : [
    { "year" : 2014, "number" : 4788 },
    { "year" : 2015, "number" : 4821 },
    { "year" : 2016, "number" : 6550 },
    { "year" : 2017, "number" : 6125 }
  ]
}
])


db.courses.insert_many([
{
  "university" : "USAL",
  "name" : "Computer Science",
  "level" : "Excellent"
},
{
  "university" : "USAL",
  "name" : "Electronics",
  "level" : "Intermediate"
},
{
  "university" : "USAL",
  "name" : "Communication",
  "level" : "Excellent"
}
])

# Aggregation stages

## MongoDB `$match`

In [7]:
cursor = db.universities.aggregate([
  { "$match" : { "country" : "Spain", "city" : "Salamanca" } }
])

print_cursor(cursor)


{'_id': ObjectId('6127a91e40d46ed59dbbc25c'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}]}

{'_id': ObjectId('6127a91e40d46ed59dbbc25d'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA', 'location': {'type': 'Point', 'coordinates': [-5.6691191, 17, 40.9631732]}, 'students': [{'year': 2014, 'number': 4788}, {'year': 2015, 'number': 4821}, {'year': 2016, 'number': 6550}, {'year': 2017, 'number': 6125}]}

#Results: 2


## MongoDB `$project`

In [8]:
cursor = db.universities.aggregate([
  { "$project" : { "_id" : 0, "country" : 1, "city" : 1, "name" : 1 } }
])

print_cursor(cursor)


{'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL'}

{'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA'}

#Results: 2


## MongoDB `$group`

In [22]:
cursor = db.universities.aggregate([
  { "$group" : { "_id" : "$name", "totaldocs" : { "$sum" : 1 } } }
])

print_cursor(cursor)


{'_id': 'USAL', 'totaldocs': 1}

{'_id': 'UPSA', 'totaldocs': 1}

#Results: 2


## MongoDB `$out`

In [23]:
db.universities.aggregate([
  { "$group" : { "_id" : "$name", "totaldocs" : { "$sum" : 1 } } },
  { "$out" : "aggResults" }
])

cursor = db.aggResults.find()
print_cursor(cursor)


{'_id': 'UPSA', 'totaldocs': 1}

{'_id': 'USAL', 'totaldocs': 1}

#Results: 2


## MongoDB `$unwind`

In [24]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" }
])

print_cursor(cursor)


{'_id': ObjectId('6127aee240d46ed59dbbc272'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2014, 'number': 24774}}

{'_id': ObjectId('6127aee240d46ed59dbbc272'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2015, 'number': 23166}}

{'_id': ObjectId('6127aee240d46ed59dbbc272'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2016, 'number': 21913}}

{'_id': ObjectId('6127aee240d46ed59dbbc272'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2017, 'number': 21715}}

#Results: 4


## MongoDB `$sort`

In [25]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } }
])

print_cursor(cursor)


{'students': {'year': 2014, 'number': 24774}}

{'students': {'year': 2015, 'number': 23166}}

{'students': {'year': 2016, 'number': 21913}}

{'students': {'year': 2017, 'number': 21715}}

#Results: 4


## MongoDB `$limit`

In [26]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } },
  { "$limit" : 2 }
])

print_cursor(cursor)


{'students': {'year': 2014, 'number': 24774}}

{'students': {'year': 2015, 'number': 23166}}

#Results: 2


## MongoDB `$addFields`

In [27]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$addFields" : { "foundation_year" : 1218 } }
])

print_cursor(cursor)


{'_id': ObjectId('6127aee240d46ed59dbbc272'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'foundation_year': 1218}

#Results: 1


## MongoDB `$count`

In [28]:
cursor = db.universities.aggregate([
  { "$unwind" : "$students" },
  { "$count" : "total_documents" }
])

print_cursor(cursor)


{'total_documents': 8}

#Results: 1


## MongoDB `$lookup`

If you want this query to run fast, you are going to need to index the `name` field in the `universities` collection and the `university` field in the `courses` collection.

In [29]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$project" : { "_id" : 0, "name" : 1 } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)


{'name': 'USAL', 'courses': [{'_id': ObjectId('6127aee240d46ed59dbbc274'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('6127aee240d46ed59dbbc275'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('6127aee240d46ed59dbbc276'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}

#Results: 1


## MongoDB `$sortByCount`

This stage is a shortcut for grouping, counting and then sorting in descending order the number of different values in a field.

In [30]:
cursor = db.courses.aggregate([
  { "$sortByCount" : "$level" }
])

print_cursor(cursor)


{'_id': 'Excellent', 'count': 2}

{'_id': 'Intermediate', 'count': 1}

#Results: 2


## MongoDB `$facet`

Sometimes when creating a report on data, you find that you need to do the same preliminary processing for a number of reports, and you are faced with having to create and maintain an intermediate collection.

In [31]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)


{'_id': ObjectId('6127aee240d46ed59dbbc272'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'courses': [{'_id': ObjectId('6127aee240d46ed59dbbc274'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('6127aee240d46ed59dbbc275'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('6127aee240d46ed59dbbc276'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}

#Results: 1


In [36]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
])

print_cursor(cursor)


{'_id': 'Excellent', 'count': 2}

{'_id': 'Intermediate', 'count': 1}

#Results: 2


In [40]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
])

print_cursor(cursor)


{'students': {'year': 2017, 'number': 21715}}

#Results: 1


In [41]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
  { "$facet" : {
      "countingLevels" :
      [
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
      ],
      "yearWithLessStudents" :
      [
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
      ]
  } }
])

print_cursor(cursor)


{'countingLevels': [{'_id': 'Excellent', 'count': 2}, {'_id': 'Intermediate', 'count': 1}], 'yearWithLessStudents': [{'students': {'year': 2017, 'number': 21715}}]}

#Results: 1


---

# Dataset 2

Use `mongoimport` to load http://media.mongodb.org/zips.json into your mongod instance.


> 
**1) put file in specific dir (desktop) and move to it "cd desktop"<br>
2) "D:\AI_PRO\Programs\MongoDB\2-Tools\setup\bin\mongoimport.exe" -d=zips -c=zipcodes zips.json<br>
..........directory of mongoimport.......................database name......collection name.......file name............**

In [42]:
db = client.zips

In [43]:
cursor = db.zipcodes.find({"_id" : "47906"})
print_cursor(cursor)


{'_id': '47906', 'city': 'WEST LAFAYETTE', 'loc': [-86.923661, 40.444025], 'pop': 54702, 'state': 'IN'}

#Results: 1


# Aggregation Pipeline

### Return States with Populations above 10 Million

`SELECT state, SUM(pop) AS totalPop
FROM zipcodes
GROUP BY state
HAVING totalPop >= (10*1000*1000)`

In [44]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": "$state", "totalPop": { "$sum": "$pop" } } },
   { "$match": { "totalPop": { "$gte": 10*1000*1000 } } }
] )

print_cursor(cursor)


{'_id': 'TX', 'totalPop': 16984601}

{'_id': 'OH', 'totalPop': 10846517}

{'_id': 'IL', 'totalPop': 11427576}

{'_id': 'PA', 'totalPop': 11881643}

{'_id': 'NY', 'totalPop': 17990402}

{'_id': 'FL', 'totalPop': 12686644}

{'_id': 'CA', 'totalPop': 29754890}

#Results: 7


### Return Average City Population by State

In [45]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
   { "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" } } }
] )

print_cursor(cursor)


{'_id': 'GA', 'avgCityPop': 11547.62210338681}

{'_id': 'VT', 'avgCityPop': 2315.8765432098767}

{'_id': 'WI', 'avgCityPop': 7323.00748502994}

{'_id': 'OK', 'avgCityPop': 6155.743639921722}

{'_id': 'MO', 'avgCityPop': 5672.195338512764}

{'_id': 'AZ', 'avgCityPop': 20591.16853932584}

{'_id': 'WA', 'avgCityPop': 12258.670025188916}

{'_id': 'WY', 'avgCityPop': 3384.5373134328356}

{'_id': 'HI', 'avgCityPop': 15831.842857142858}

{'_id': 'NV', 'avgCityPop': 18209.590909090908}

{'_id': 'UT', 'avgCityPop': 9518.508287292818}

{'_id': 'IN', 'avgCityPop': 9271.130434782608}

{'_id': 'DC', 'avgCityPop': 303450.0}

{'_id': 'SD', 'avgCityPop': 1839.6746031746031}

{'_id': 'WV', 'avgCityPop': 2771.4775888717154}

{'_id': 'MI', 'avgCityPop': 12087.512353706112}

{'_id': 'TX', 'avgCityPop': 13775.02108678021}

{'_id': 'OH', 'avgCityPop': 12700.839578454332}

{'_id': 'TN', 'avgCityPop': 9656.350495049504}

{'_id': 'ID', 'avgCityPop': 4320.811158798283}

{'_id': 'IL', 'avgCityPop': 9954.3344947

### Return Largest and Smallest Cities by State

In [46]:
cursor = db.zipcodes.aggregate( [
   { "$group":
      {
        "_id": { "state": "$state", "city": "$city" },
        "pop": { "$sum": "$pop" }
      }
   },
   { "$sort": { "pop": 1 } },
   { "$group":
      {
        "_id" : "$_id.state",
        "biggestCity":  { "$last": "$_id.city" },
        "biggestPop":   { "$last": "$pop" },
        "smallestCity": { "$first": "$_id.city" },
        "smallestPop":  { "$first": "$pop" }
      }
   },
  { "$project":
    { "_id": 0,
      "state": "$_id",
      "biggestCity":  { "name": "$biggestCity",  "pop": "$biggestPop" },
      "smallestCity": { "name": "$smallestCity", "pop": "$smallestPop" }
    }
  }
] )

print_cursor(cursor)


{'biggestCity': {'name': 'PHILADELPHIA', 'pop': 1610956}, 'smallestCity': {'name': 'HAMILTON', 'pop': 0}, 'state': 'PA'}

{'biggestCity': {'name': 'DES MOINES', 'pop': 148155}, 'smallestCity': {'name': 'DOUDS', 'pop': 15}, 'state': 'IA'}

{'biggestCity': {'name': 'LOUISVILLE', 'pop': 288058}, 'smallestCity': {'name': 'BROWDER', 'pop': 0}, 'state': 'KY'}

{'biggestCity': {'name': 'CHICAGO', 'pop': 2452177}, 'smallestCity': {'name': 'ANCONA', 'pop': 38}, 'state': 'IL'}

{'biggestCity': {'name': 'JACKSON', 'pop': 204788}, 'smallestCity': {'name': 'CHUNKY', 'pop': 79}, 'state': 'MS'}

{'biggestCity': {'name': 'BOISE', 'pop': 165522}, 'smallestCity': {'name': 'KEUTERVILLE', 'pop': 0}, 'state': 'ID'}

{'biggestCity': {'name': 'WORCESTER', 'pop': 169856}, 'smallestCity': {'name': 'BUCKLAND', 'pop': 16}, 'state': 'MA'}

{'biggestCity': {'name': 'MEMPHIS', 'pop': 632837}, 'smallestCity': {'name': 'ALLRED', 'pop': 2}, 'state': 'TN'}

{'biggestCity': {'name': 'CLEVELAND', 'pop': 536759}, 'smalle

---